In [1]:
#general analysis packages
import numpy as np
import pandas as pd
from importlib import reload
import dash_radial_decoding as drd

In [2]:
location_path = "/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dots_comb/ADCG/Pos_0/locations_z_0.csv"
codebook_path = "/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/barcode_key/codebook_string.csv"
output_dir_new = "/groups/CaiLab/personal/Lex/data_analysis_code/decoding_files/DASH_Radial_Decoding/development"

In [3]:
codebook = pd.read_csv(codebook_path, index_col=0)
location = pd.read_csv(location_path)

In [4]:
location

,Unnamed: 0,hyb,ch,x,y,z,size,intensity
0,0,0,1,35.496967,47.474342,0,1.251960,4735.511773
1,1,0,1,37.507759,39.487866,0,1.321371,2981.720575
2,2,0,1,2.752118,29.119482,0,1.364721,2449.489442
3,3,0,1,18.588521,37.024593,0,1.380081,1699.325928
4,4,0,1,47.567539,48.493810,0,1.207778,2984.144538
...,...,...,...,...,...,...,...,...
2872278,6205968,79,1,766.602554,836.677924,0,1.600000,1596.274052
2872279,6205969,79,1,767.730267,0.990257,0,1.200000,2341.968393
2872280,6205970,79,1,1153.362498,1949.623801,0,1.600000,2517.086264
2872281,6205971,79,1,1279.595584,958.915639,0,1.600000,1001.672175


In [7]:
import time
start = time.time()

dot_idx,ambiguity_scores,codeword_score_list,total_distance_list = drd.radial_decoding(
    location, codebook, n_neighbors=4,
    num_barcodes = 4, radius=1,diff=1,
    seed=0, hybs = 80)

print("This task took=", round((time.time()-start)/60, 2), "minutes :)")

This task took= 13.21 minutes :)


In [8]:
filtered_set = [set(i) for i in dot_idx]

In [9]:
#group all sets with any matching dot indicies
matching_element_list = []
#make a copy of list for searching
search_set = filtered_set.copy()
#keep history of elements
history_element = set()
#lets group
for _set in filtered_set:
    temp=[]
    if _set & history_element:
        continue
    for _search in search_set:
        if _set & _search:
            temp.append(_search)
    for item in np.unique(list(_set)):
        history_element.add(item)
    matching_element_list.append(temp)

KeyboardInterrupt: 

In [10]:
#isolate only lists > 1
filter_list = []
for sublist in matching_element_list:
    if len(sublist) > 1:
        filter_list.append(sublist)

In [11]:
#generate hash table for codeword score
codeword_hash = {}
for i in range(len(codeword_score_list)):
    #if the key exists, add the codeword score to existing key
    if tuple(sorted(codeword_score_list[i][0])) in codeword_hash:
        new_value = codeword_hash[tuple(sorted(codeword_score_list[i][0]))] + codeword_score_list[i][1]
        codeword_hash.update({tuple(sorted(codeword_score_list[i][0])):new_value})
    else:
        codeword_hash.update({tuple(sorted(codeword_score_list[i][0])):codeword_score_list[i][1]})

In [12]:
#generate hash table for total distance
distance_hash = {}
for i in range(len(total_distance_list)):
    #if the key exists, add the total distance to existing key
    if tuple(sorted(total_distance_list[i][0])) in distance_hash:
        new_value = distance_hash[tuple(sorted(total_distance_list[i][0]))] + total_distance_list[i][1]
        distance_hash.update({tuple(sorted(total_distance_list[i][0])):new_value})
    else:
        distance_hash.update({tuple(sorted(total_distance_list[i][0])):total_distance_list[i][1]})

In [13]:
def score_codewords(code_list,codeword_hash, distance_hash):
    """
    This function will pick best overall codeword
    Parameters
    ----------
    code_list: list of codword sets we are trying to compare
    codeword_hash: a hash table to return overall codeword score
    distance_hash: a hash table to return overall distance for codeword
    """
    #keep score and distance for each dot sequence
    track_score = []
    track_distance = []
    for codeword in code_list:
        track_score.append(codeword_hash[tuple(sorted(codeword))])
        track_distance.append(distance_hash[tuple(sorted(codeword))])
    #check if there are equal scores
    #if that is the case use pure distance (taking the shortest)
    if len(np.where(track_score == np.amax(track_score))[0]) >= 2:
        best_code = code_list[np.argmin(track_distance)]
    else:
        best_code = code_list[np.argmax(track_score)]
    
    return best_code

def recover_codes(code_list, best_codeword):
    """
    This function will try to recover codewords if the dots selected 
    from score_codewords() function is not repeated in the other choices
    Parameters
    ----------
    code_list: list of codeword sets we analyzed from score_codewords()
    best_codeword: the best codeword set chosen from score_codewords()
    """
    #store winner history
    winner_history = set()
    for unique_dots in best_codeword:
        winner_history.add(unique_dots)
    #recover codewords if the winner dot sequence does not appear in other choices
    recovered_codes = []
    for i in range(len(code_list)):
        if code_list[i] & winner_history:
            continue
        else:
            recover_code = code_list[i]
            recovered_codes.append(recover_code)
            
    return recovered_codes

In [16]:
#pick the best final codewords
final_codewords = []
for codes in filter_list:
    #get best code word list of sets
    best_code = score_codewords(codes,codeword_hash, distance_hash)
    #store winner
    final_codewords.append(best_code)
#     #check to see if we can recover any codewords
#     recovered_codes = recover_codes(codes, best_code)
#     #if there is one returned, we will add to final
#     if len(recovered_codes) == 1:
#         final_codewords.append(recovered_codes[0])
#     #if nothing is returned then we will continue
#     elif len(recovered_codes) == 0:
#         continue
#     #if there is multiple choices we will perform a while loop until the choices become 0
#     else:
#         while len(recovered_codes) != 0:
#             #find the best codeword in recovered list
#             best_2 = score_codewords(recovered_codes,codeword_hash, distance_hash)
#             final_codewords.append(best_2)
#             #check to see if we can recover any codewords again
#             recovered_2 = recover_codes(recovered_codes, best_2)
#             #if there is one choice returned we can add
#             #otherwise we will repeat loop or end loop
#             if len(recovered_2) == 1:
#                 final_codewords.append(recovered_2[0])
#             #reassign recovered_codes
#             recovered_codes = recovered_2            

In [15]:
len(final_codewords)

2773

In [17]:
len(final_codewords)

2436